In [1]:
pip install --upgrade langchain

   ---------------------------------------- 0.0/803.6 kB ? eta -:--:--
   --------------------------------------- 803.6/803.6 kB 17.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------- ----- 1.4/1.6 MB 29.4 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 25.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.2
    Uninstalling langchain-community-0.0.2:
      Successfully uninstalled langchain-community-0.0.2
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.352
    Uninstalling langchain-0.0.352:
      Successfully uninstalled langchain-0.0.352
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
embedchain 0.1.31 requires langchain<0.0.337,>=0.0.336, but you have langchain 0.1.4 which is incompatible.
embedchain 0.1.31 requires tiktoken<0.5.0,>=0.4.0, but you have tiktoken 0.5.2 which is incompatible.
jupyter-ai 2.0.1 requires importlib-metadata~=5.2.0, but you have importlib-metadata 6.0.1 which is incompatible.
jupyter-ai 2.0.1 requires langchain==0.0.220, but you have langchain 0.1.4 which is incompatible.
jupyter-ai 2

In [32]:
from langchain_community.document_loaders.s3_file import S3FileLoader

# MinIO Configuration
endpoint = 'play.min.io:9000'
access_key = 'minioadmin'
secret_key = 'minioadmin'
use_ssl = True

# Initialize and load a single document
file_loader = S3FileLoader(
    bucket='web-documentation',
    key='MinIO_Quickstart.md',
    endpoint_url=f'http{"s" if use_ssl else ""}://{endpoint}',
    aws_access_key_id=access_key, 
    aws_secret_access_key=secret_key, 
    use_ssl=use_ssl
)

document = file_loader.load()
document

[Document(page_content='MinIO Quickstart Guide\n\nMinIO is a High Performance Object Storage released under GNU Affero General Public License v3.0. It is API compatible with Amazon S3 cloud storage service. Use MinIO to build high performance infrastructure for machine learning, analytics and application data workloads.\n\nThis README provides quickstart instructions on running MinIO on bare metal hardware, including container-based installations. For Kubernetes environments, use the MinIO Kubernetes Operator.\n\nContainer Installation\n\nUse the following commands to run a standalone MinIO server as a container.\n\nStandalone MinIO servers are best suited for early development and evaluation. Certain features such as versioning, object locking, and bucket replication\nrequire distributed deploying MinIO with Erasure Coding. For extended development and production, deploy MinIO with Erasure Coding enabled - specifically,\nwith a minimum of 4 drives per MinIO server. See MinIO Erasure C

In [33]:
from langchain_community.document_loaders.s3_directory import S3DirectoryLoader

# Initialize and load documents
directory_loader = S3DirectoryLoader(
    bucket='web-documentation',
    prefix='',
    endpoint_url=f'http{"s" if use_ssl else ""}://{endpoint}',
    aws_access_key_id=access_key, 
    aws_secret_access_key=secret_key, 
    use_ssl=use_ssl
)

documents = directory_loader.load()
documents

[Document(page_content='MinIO Quickstart Guide\n\nMinIO is a High Performance Object Storage released under GNU Affero General Public License v3.0. It is API compatible with Amazon S3 cloud storage service. Use MinIO to build high performance infrastructure for machine learning, analytics and application data workloads.\n\nThis README provides quickstart instructions on running MinIO on bare metal hardware, including container-based installations. For Kubernetes environments, use the MinIO Kubernetes Operator.\n\nContainer Installation\n\nUse the following commands to run a standalone MinIO server as a container.\n\nStandalone MinIO servers are best suited for early development and evaluation. Certain features such as versioning, object locking, and bucket replication\nrequire distributed deploying MinIO with Erasure Coding. For extended development and production, deploy MinIO with Erasure Coding enabled - specifically,\nwith a minimum of 4 drives per MinIO server. See MinIO Erasure C

In [34]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
import os

# Set your OpenAI API key
os.environ['OPENAI_API_KEY'] = 'your-openai-api-key-here'
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")



prompt = ChatPromptTemplate.from_template(
    "Summarize the following document '{document_name}':{context}Please provide the summary and key points."
)

# Assuming 'documents' and 'document' are lists of document objects
loaded_documents = [documents, document]  # From S3 Loaders
flattened_documents = [doc for sublist in loaded_documents for doc in sublist]

for loaded_document in flattened_documents:
    document_text = loaded_document.page_content
    # Assuming each document has a 'name' attribute
    document_name = getattr(loaded_document, 'name', 'Unknown Document')  
    chain = (
        RunnableLambda(lambda x: {"context": document_text, "document_name": document_name})
        | prompt
        | model
        | StrOutputParser()
    )
    summary = chain.invoke(None)
    print("Summary:", summary)

Summary: The document is a quickstart guide for setting up and running MinIO, a high-performance object storage system. MinIO is API compatible with Amazon S3 and can be used to build infrastructure for machine learning, analytics, and application data workloads.

The guide provides instructions for installing and running MinIO on different platforms, including container-based installations, macOS, GNU/Linux, Microsoft Windows, and from source. It also includes recommendations for deploying MinIO in production environments and instructions for testing MinIO connectivity using the MinIO Console and MinIO Client (mc).

The document emphasizes the importance of deploying MinIO with erasure coding for extended development and production, and provides links to additional documentation for MinIO SDKs and other resources.

The guide also includes instructions for upgrading MinIO and provides a checklist for performing upgrades in a safe and efficient manner.

Overall, the document provides a 